In [2]:
import pandas as pd
import numpy as np
import sys
import sklearn
import tensorflow as tf
print(pd.__version__)
print(np.__version__)
print(sys.version)
print(sklearn.__version__)
print(tf.__version__)

2.2.2
1.26.4
3.12.3 (tags/v3.12.3:f6650f9, Apr  9 2024, 14:05:25) [MSC v.1938 64 bit (AMD64)]
1.4.2
2.16.1


In [1]:
col_names = ["duration","protocol_type","service","flag","src_bytes",
    "dst_bytes","land","wrong_fragment","urgent","hot","num_failed_logins",
    "logged_in","num_compromised","root_shell","su_attempted","num_root",
    "num_file_creations","num_shells","num_access_files","num_outbound_cmds",
    "is_host_login","is_guest_login","count","srv_count","serror_rate",
    "srv_serror_rate","rerror_rate","srv_rerror_rate","same_srv_rate",
    "diff_srv_rate","srv_diff_host_rate","dst_host_count","dst_host_srv_count",
    "dst_host_same_srv_rate","dst_host_diff_srv_rate","dst_host_same_src_port_rate",
    "dst_host_srv_diff_host_rate","dst_host_serror_rate","dst_host_srv_serror_rate",
    "dst_host_rerror_rate","dst_host_srv_rerror_rate","label"]

# KDDTrain+_2.csv & KDDTest+_2.csv are the datafiles without the last column about the difficulty score
# these have already been removed.
df = pd.read_csv("C:\\Users\\91630\\Desktop\\subjects pdf\\KDDTrain+.csv", header=None, names = col_names)
df_test = pd.read_csv("C:\\Users\\91630\Desktop\\subjects pdf\\KDDTest+.csv", header=None, names = col_names)

# shape, this gives the dimensions of the dataset
print('Dimensions of the Training set:',df.shape)
print('Dimensions of the Test set:',df_test.shape)

<>:15: SyntaxWarning: invalid escape sequence '\D'
<>:15: SyntaxWarning: invalid escape sequence '\D'
C:\Users\91630\AppData\Local\Temp\ipykernel_10324\1092257020.py:15: SyntaxWarning: invalid escape sequence '\D'
  df_test = pd.read_csv("C:\\Users\\91630\Desktop\\subjects pdf\\KDDTest+.csv", header=None, names = col_names)
C:\Users\91630\AppData\Local\Temp\ipykernel_10324\1092257020.py:15: SyntaxWarning: invalid escape sequence '\D'
  df_test = pd.read_csv("C:\\Users\\91630\Desktop\\subjects pdf\\KDDTest+.csv", header=None, names = col_names)


NameError: name 'pd' is not defined

In [10]:
#Identify categorical features
# colums that are categorical and not binary yet: protocol_type (column 2), service (column 3), flag (column 4).
# explore categorical features
print('Training set:')
for col_name in df.columns:
    if df[col_name].dtypes == 'object' :
        unique_cat = len(df[col_name].unique())
        print("Feature '{col_name}' has {unique_cat} categories".format(col_name=col_name, unique_cat=unique_cat))

#see how distributed the feature service is, it is evenly distributed and therefore we need to make dummies for all.
print()
print('Distribution of categories in protocol type:')
print(df['protocol_type'].value_counts().sort_values(ascending=False).head())
print('Distribution of categories in service type:')
print(df['service'].value_counts().sort_values(ascending=False).head())
print('Distribution of categories in flag type:')
print(df['flag'].value_counts().sort_values(ascending=False).head())
print('Distribution of categories in label type:')
print(df['label'].value_counts().sort_values(ascending=False).head())

Training set:
Feature 'protocol_type' has 3 categories
Feature 'service' has 70 categories
Feature 'flag' has 11 categories
Feature 'label' has 23 categories

Distribution of categories in protocol type:
protocol_type
tcp     102689
udp      14993
icmp      8291
Name: count, dtype: int64
Distribution of categories in service type:
service
http        40338
private     21853
domain_u     9043
smtp         7313
ftp_data     6860
Name: count, dtype: int64
Distribution of categories in flag type:
flag
SF      74945
S0      34851
REJ     11233
RSTR     2421
RSTO     1562
Name: count, dtype: int64
Distribution of categories in label type:
label
normal       67343
neptune      41214
satan         3633
ipsweep       3599
portsweep     2931
Name: count, dtype: int64


In [11]:
print('Test set:')
for col_name in df_test.columns:
    if df_test[col_name].dtypes == 'object' :
        unique_cat = len(df_test[col_name].unique())
        print("Feature '{col_name}' has {unique_cat} categories".format(col_name=col_name, unique_cat=unique_cat))

Test set:
Feature 'protocol_type' has 3 categories
Feature 'service' has 63 categories
Feature 'flag' has 11 categories
Feature 'label' has 38 categories


In [12]:
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
categorical_columns=['protocol_type', 'service', 'flag']
# insert code to get a list of categorical columns into a variable, categorical_columns
categorical_columns=['protocol_type', 'service', 'flag'] 
 # Get the categorical values into a 2D numpy array
df_categorical_values = df[categorical_columns]
testdf_categorical_values = df_test[categorical_columns]
df_categorical_values.head()

,protocol_type,service,flag
0,tcp,ftp_data,SF
1,udp,other,SF
2,tcp,private,S0
3,tcp,http,SF
4,tcp,http,SF


In [13]:
df_categorical_values_enc=df_categorical_values.apply(LabelEncoder().fit_transform)
print(df_categorical_values_enc.head())
# test set
testdf_categorical_values_enc=testdf_categorical_values.apply(LabelEncoder().fit_transform)

   protocol_type  service  flag
0              1       20     9
1              2       44     9
2              1       49     5
3              1       24     9
4              1       24     9


In [14]:
df_categorical_values_enc.head(5)

,protocol_type,service,flag
0,1,20,9
1,2,44,9
2,1,49,5
3,1,24,9
4,1,24,9


In [15]:
df.drop('flag', axis=1, inplace=True)
df.drop('protocol_type', axis=1, inplace=True)
df.drop('service', axis=1, inplace=True)
df=df.join(df_categorical_values_enc)
#print(newdf)
# test data
df_test.drop('flag', axis=1, inplace=True)
df_test.drop('protocol_type', axis=1, inplace=True)
df_test.drop('service', axis=1, inplace=True)
df_test=df_test.join(testdf_categorical_values_enc)
print(df.head())
print(df.shape)
#print(newdf_test)
print(df_test.shape)

   duration  src_bytes  dst_bytes  land  wrong_fragment  urgent  hot  \
0         0        491          0     0               0       0    0   
1         0        146          0     0               0       0    0   
2         0          0          0     0               0       0    0   
3         0        232       8153     0               0       0    0   
4         0        199        420     0               0       0    0   

   num_failed_logins  logged_in  num_compromised  ...  \
0                  0          0                0  ...   
1                  0          0                0  ...   
2                  0          0                0  ...   
3                  0          1                0  ...   
4                  0          1                0  ...   

   dst_host_same_src_port_rate  dst_host_srv_diff_host_rate  \
0                         0.17                         0.00   
1                         0.88                         0.00   
2                         0.00     

In [16]:
# take label column
labeldf=df['label']
labeldf_test=df_test['label']

# change the label column
newlabeldf=labeldf.replace({ 'normal' : 4, 'neptune' : 0 ,'back': 0, 'land': 0, 'pod': 0, 'smurf': 0, 'teardrop': 0,'mailbomb': 0, 'apache2': 0, 'processtable': 0, 'udpstorm': 0, 'worm': 0,
                           'ipsweep' : 1,'nmap' : 1,'portsweep' : 1,'satan' : 1,'mscan' : 1,'saint' : 1
                           ,'ftp_write': 2,'guess_passwd': 2,'imap': 2,'multihop': 2,'phf': 2,'spy': 2,'warezclient': 2,'warezmaster': 2,'sendmail': 2,'named': 2,'snmpgetattack': 2,'snmpguess': 2,'xlock': 2,'xsnoop': 2,'httptunnel': 2,
                           'buffer_overflow': 3,'loadmodule': 3,'perl': 3,'rootkit': 3,'ps': 3,'sqlattack': 3,'xterm': 3})
newlabeldf_test=labeldf_test.replace({ 'normal' : 4, 'neptune' : 0 ,'back': 0, 'land': 0, 'pod': 0, 'smurf': 0, 'teardrop': 0,'mailbomb': 0, 'apache2': 0, 'processtable': 0, 'udpstorm': 0, 'worm': 0,
                           'ipsweep' : 1,'nmap' : 1,'portsweep' : 1,'satan' : 1,'mscan' : 1,'saint' : 1
                           ,'ftp_write': 2,'guess_passwd': 2,'imap': 2,'multihop': 2,'phf': 2,'spy': 2,'warezclient': 2,'warezmaster': 2,'sendmail': 2,'named': 2,'snmpgetattack': 2,'snmpguess': 2,'xlock': 2,'xsnoop': 2,'httptunnel': 2,
                           'buffer_overflow': 3,'loadmodule': 3,'perl': 3,'rootkit': 3,'ps': 3,'sqlattack': 3,'xterm': 3})
# put the new label column back
print(newlabeldf.head())
df['label'] = newlabeldf
df_test['label'] = newlabeldf_test
print(df['label'].head())
print(df.head())
print(df_test.shape)

0    4
1    4
2    0
3    4
4    4
Name: label, dtype: int64
0    4
1    4
2    0
3    4
4    4
Name: label, dtype: int64
   duration  src_bytes  dst_bytes  land  wrong_fragment  urgent  hot  \
0         0        491          0     0               0       0    0   
1         0        146          0     0               0       0    0   
2         0          0          0     0               0       0    0   
3         0        232       8153     0               0       0    0   
4         0        199        420     0               0       0    0   

   num_failed_logins  logged_in  num_compromised  ...  \
0                  0          0                0  ...   
1                  0          0                0  ...   
2                  0          0                0  ...   
3                  0          1                0  ...   
4                  0          1                0  ...   

   dst_host_same_src_port_rate  dst_host_srv_diff_host_rate  \
0                         0.17         

C:\Users\91630\AppData\Local\Temp\ipykernel_21308\549405909.py:6: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  newlabeldf=labeldf.replace({ 'normal' : 4, 'neptune' : 0 ,'back': 0, 'land': 0, 'pod': 0, 'smurf': 0, 'teardrop': 0,'mailbomb': 0, 'apache2': 0, 'processtable': 0, 'udpstorm': 0, 'worm': 0,
C:\Users\91630\AppData\Local\Temp\ipykernel_21308\549405909.py:10: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  newlabeldf_test=labeldf_test.replace({ 'normal' : 4, 'neptune' : 0 ,'back': 0, 'land': 0, 'pod': 0, 'smurf': 0, 'teardrop

In [17]:
data=df
data = data.drop("label", axis=1)
data.shape

(125973, 41)

In [18]:
data_test=df_test
data_test= data_test.drop("label", axis=1)
data_test.shape

(22543, 41)

In [19]:
from sklearn.preprocessing import MinMaxScaler

In [20]:
scaler =MinMaxScaler()
scaled_data =scaler.fit_transform(data)

In [21]:
scaler =MinMaxScaler()
scaled_data_test =scaler.fit_transform(data_test)

In [22]:
df=scaled_data
df_test=scaled_data_test

In [23]:
df.shape

(125973, 41)

In [24]:
df_test.shape

(22543, 41)

In [25]:
df_data=np.column_stack((df,newlabeldf))

In [26]:
df_data=np.column_stack((df,newlabeldf))

In [27]:
df_data_test=np.column_stack((df_test,newlabeldf_test))

In [28]:
df_data

array([[0.00000000e+00, 3.55806412e-07, 0.00000000e+00, ...,
        2.89855072e-01, 9.00000000e-01, 4.00000000e+00],
       [0.00000000e+00, 1.05799870e-07, 0.00000000e+00, ...,
        6.37681159e-01, 9.00000000e-01, 4.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        7.10144928e-01, 5.00000000e-01, 0.00000000e+00],
       ...,
       [0.00000000e+00, 1.61670897e-06, 2.93143779e-07, ...,
        7.82608696e-01, 9.00000000e-01, 4.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        4.34782609e-01, 5.00000000e-01, 0.00000000e+00],
       [0.00000000e+00, 1.09423153e-07, 0.00000000e+00, ...,
        2.89855072e-01, 9.00000000e-01, 4.00000000e+00]])

In [29]:
df_data_test.shape

(22543, 42)

In [37]:
from __future__ import print_function
import numpy as np
np.random.seed(1337)  # for reproducibility
import keras
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Lambda
from keras.layers import Embedding
from keras.layers import Convolution1D,MaxPooling1D, Flatten
from keras.datasets import imdb
from keras import backend as K
#from sklearn.cross_validation import train_test_split
import pandas as pd
from keras.utils import to_categorical

from sklearn.preprocessing import Normalizer
from keras.models import Sequential
from keras.layers import Convolution1D, Dense, Dropout, Flatten, MaxPooling1D
from keras  import utils
import numpy as np
import h5py
from keras import callbacks
from keras.layers import LSTM, GRU, SimpleRNN
from keras.callbacks import CSVLogger
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, CSVLogger

from keras.optimizers import Adam


traindata = df_data
testdata = df_data_test


X = traindata[:,1:42]
Y = traindata[:,41]
C = testdata[:,41]
T = testdata[:,1:42]

scaler = Normalizer().fit(X)
trainX = scaler.transform(X)
   
scaler = Normalizer().fit(T)
testT = scaler.transform(T)

y_train = np.array(Y)
y_test = np.array(C)


#y_train= to_categorical(y_train1)
#y_test= to_categorical(y_test1)

# reshape input to be [samples, time steps, features]
X_train = np.reshape(trainX, (trainX.shape[0],trainX.shape[1]))
X_test = np.reshape(testT, (testT.shape[0],testT.shape[1]))


num_classes = 5

y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)



batch_size = 100


dnn = Sequential()

dnn.add(Dense(64, activation='relu',input_shape=(41,)))
dnn.add(Dense(64, activation='relu'))
dnn.add(Dense(64, activation='relu'))
#model.add(Dense(5)
dnn.add(Dense(5, activation="softmax"))          
#model.add(Activation('softmax'))
opt = Adam(lr=0.01)



# define optimizer and objective, compile cnn

dnn.compile(loss="binary_crossentropy", optimizer=opt,metrics=['accuracy'])

# train
checkpointer = callbacks.ModelCheckpoint(filepath="D:/Results/II/NSLKDD/DNN-L3/checkpoint-{epoch:02d}.hdf5", verbose=1, save_best_only=True, monitor='val_acc',mode='max')
csv_logger = CSVLogger('D:/Results/II/NSLKDD/DNN-L3/dnnanalysis.csv',separator=',', append=False)
#model.fit(X_train, y_train, epochs=10, verbose=1)
#model.fit(X_train, y_train, batch_size=batch_size, epochs=500, validation_data=(X_test, y_test),callbacks=[checkpointer,csv_logger])
dnn.fit(X_train, y_train, nb_epoch=50, validation_data=(X_test, y_test),callbacks=[checkpointer,csv_logger])

  
dnn.save("D:/Results/II/NSLKDD/DNN-L3/dnn_model.hdf5")

loss, accuracy = dnn.evaluate(X_test,y_test)
print("\nLoss: %.2f, Accuracy: %.2f%%" % (loss, accuracy*100))
y_pred = dnn.predict_classes(X_test)





c:\Users\91630\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\dense.py:86: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


ValueError: Argument(s) not recognized: {'lr': 0.01}

In [50]:
from __future__ import print_function
import numpy as np
np.random.seed(1337)  # for reproducibility

from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Lambda
from keras.layers import Embedding
from keras.layers import Convolution1D,MaxPooling1D, Flatten
from keras.datasets import imdb
from keras import backend as K
#from sklearn.cross_validation import train_test_split
import pandas as pd
from keras.utils import to_categorical

from sklearn.preprocessing import Normalizer
from keras.models import Sequential
from keras.layers import Convolution1D, Dense, Dropout, Flatten, MaxPooling1D
from keras import utils
import numpy as np
import h5py
from keras import callbacks
from keras.layers import LSTM, GRU, SimpleRNN
from keras.callbacks import CSVLogger
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, CSVLogger


#traindata = pd.read_csv('kdd/multiclass/kddtrain.csv', header=None)
testdata = df_data_test


#X = traindata.iloc[:,0:42]
#Y = traindata.iloc[:,0]
C = testdata[:,41]
T = testdata[:,1:42]

#scaler = Normalizer().fit(X)
#trainX = scaler.transform(X)

scaler = Normalizer().fit(T)
testT = scaler.transform(T)

#y_train1 = np.array(Y)
y_test= np.array(C)


#y_train= to_categorical(y_train1)
#y_test= to_categorical(y_test1)

# reshape input to be [samples, time steps, features]
#X_train = np.reshape(trainX, (trainX.shape[0],trainX.shape[1],1))
X_test = np.reshape(testT, (testT.shape[0],testT.shape[1]))



y_test= np.array(C)

y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)


batch_size = 100


dnn = Sequential()

dnn.add(Dense(64, activation='relu',input_shape=(41,)))
dnn.add(Dense(64, activation='relu'))
dnn.add(Dense(64, activation='relu'))
#model.add(Dense(5)
dnn.add(Dense(5, activation="softmax"))          
#model.add(Activation('softmax'))
#opt = Adam(lr=0.01)



#model.add(Activation('softmax'))
#opt=Adam(lr=0.01)


# define optimizer and objective, compile cnn
'''
cnn.compile(loss="categorical_crossentropy", optimizer="adam",metrics=['accuracy'])
# train
checkpointer = callbacks.ModelCheckpoint(filepath="results/cnn3results/checkpoint-{epoch:02d}.hdf5", verbose=1, save_best_only=True, monitor='val_acc',mode='max')
csv_logger = CSVLogger('results/cnn3results/cnntrainanalysis3.csv',separator=',', append=False)
cnn.fit(X_train, y_train, nb_epoch=1000, show_accuracy=True,validation_data=(X_test, y_test),callbacks=[checkpointer,csv_logger])
cnn.save("results/cnn3results/cnn_model.hdf5")
'''

dnn.load_weights("C:\\Users\\91630\\Desktop\\subjects pdf\\dnn_model.hdf5")

y_pred = dnn.predict_classes(X_test)

np.savetxt("C:\\Users\\91630\\Desktop\\subjects pdf\\ExceptResult.txt", y_test, fmt='%01d')
np.savetxt("C:\\Users\\91630\\Desktop\\subjects pdf\\PrediticonResult.txt", y_pred, fmt='%01d')

dnn.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
loss, accuracy = dnn.evaluate(X_test, y_test)
print("\nLoss: %.2f, Accuracy: %.2f%%" % (loss, accuracy*100))

MemoryError: Unable to allocate 14.7 GiB for an array with shape (393665625, 5) and data type float64

In [ ]:
y_score = dnn.predict_proba(X_test)

In [ ]:
y_test

In [ ]:
from scipy import interp
from itertools import cycle
from sklearn.metrics import roc_curve,auc
import numpy as np
import matplotlib.pyplot as plt

lw = 2
num_classes =5

fpr =dict()
tpr = dict()
roc_auc = dict()
for i in range(num_classes):
    fpr[i],tpr[i],_ = roc_curve(y_test[:, i], y_score[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

fpr["micro"],tpr["micro"],_ = roc_curve(y_test.ravel(), y_score.ravel())
roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])

all_fpr = np.unique(np.concatenate([fpr[i] for i in range(num_classes)]))

mean_tpr = np.zeros_like(all_fpr)
for i in range(num_classes):
    mean_tpr += interp(all_fpr, fpr[i], tpr[i])

mean_tpr /= num_classes

fpr["macro"] = all_fpr
tpr["macro"] = mean_tpr
roc_auc["macro"] = auc(fpr["macro"], tpr["macro"])

plt.figure(1)
plt.plot(fpr["micro"], tpr["micro"],
         label = 'micro-average ROC curve (area ={0:0.2f})'.format(roc_auc["micro"]),
         color = 'deeppink', linestyle=':', linewidth = 4)

plt.plot(fpr["macro"], tpr["macro"],
         label='macro-average ROC curve (area = {0:0.2f})'
               ''.format(roc_auc["macro"]),
         color='navy', linestyle=':', linewidth=4)

colors = cycle(['aqua', 'darkorange', 'cornflowerblue','red','blue'])
for i, color in zip(range(num_classes), colors):
    
    if i == 0:
        plt.plot(fpr[i], tpr[i], color=color, lw=lw,
        label='ROC curve of class DoS (area = {1:0.2f})'
        ' '.format(i, roc_auc[i]))
    if i==1:
        plt.plot(fpr[i], tpr[i], color=color, lw=lw,
        label='ROC curve of class Prob (area = {1:0.2f})'
        ''.format(i, roc_auc[i]))
    if i==2:
        plt.plot(fpr[i], tpr[i], color=color, lw=lw,
        label='ROC curve of class R2L (area = {1:0.2f})'
        ''.format(i, roc_auc[i]))
    if i==3:
        plt.plot(fpr[i], tpr[i], color=color, lw=lw,
        label='ROC curve of class U2R (area = {1:0.2f})'
        ''.format(i, roc_auc[i]))
    if i==4:
        plt.plot(fpr[i], tpr[i], color=color, lw=lw,
        label='ROC curve of class Normal (area = {1:0.2f})'
        ''.format(i, roc_auc[i]))

plt.plot([0, 1], [0, 1], 'k--', lw=lw)
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic to multi-class')
plt.legend(loc="lower right")
plt.show()

In [ ]:
predicted = np.genfromtxt ('D:/Results/II/NSLKDD/DNN-L3/res/predicted.txt', delimiter=" ")
expected = np.genfromtxt ('D:/Results/II/NSLKDD/DNN-L3/res/expected.txt', delimiter=" ")
y_true = expected[:]
y_pred = predicted[:]

In [ ]:
np.argmax(y_true,axis=1)

In [ ]:
y_true

In [ ]:
#Network-Intrusion-Detection/NSL-KDD/LSTM/multiclass/confmat.py
#import pandas as pd
#import pandas_ml as pdml

import seaborn as sn
import pandas as pd

import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc

import numpy as np
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
import mlxtend                                                          
print(mlxtend.__version__)  

#import pystan
import seaborn as sns


from mlxtend.plotting import plot_confusion_matrix

#class_names['DoS', 'Prob', 'R2L', 'U2R', 'Normal']
y_true = np.argmax(y_true,axis=1)
#print(np.count_nonzero(y_true <> 2))
cm = confusion_matrix(y_true,y_pred)
print(cm)

cm = pd.DataFrame(cm,
                     index = ['DoS','Prob','R2L','U2R','Normal'], 
                     columns = ['DoS','Prob','R2L','U2R','Normal'])



plt.figure()
sns.heatmap(cm, annot=True, fmt="d", cmap="YlGnBu")
#plt.title('CNN  \nAccuracy:{0:.3f}'.format(accuracy_score(y_test, y_pred)))
plt.ylabel('True label')
plt.xlabel('Predicted label')
plt.show()


#tn, fp, fn, tp = confusion_matrix(new_y_true, new_y_pred).ravel()

#print(" TP:\t" + str(tp) + "\nTN:\t" + str(tn) + "\nFP:\t" + str(fp) + "\nFN:\t" + str(fn))

y_true1 = np.array(y_true)
y_pred1 = np.array(y_pred)

TP=0
FP=0
TN=0
FN=0

for i in range(len(y_pred1)): 
        if y_true1[i]==y_pred1[i] and y_pred1[i]!=4:
           TP += 1
        if y_pred1[i]!=4 and y_true1[i]!=y_pred1[i]:
           FP += 1
        if y_true1[i]==y_pred1[i] and y_pred1[i]==4:
           TN += 1
        if y_true1[i]<4 and y_pred1[i]==4:
           FN += 1

print(" TP:\t" + str(TP) + "\nTN:\t" + str(TN) + "\nFP:\t" + str(FP) + "\nFN:\t" + str(FN))

print(accuracy_score(y_true1, y_pred1))
#print(cm)

TPR = TP/(TP+FN)
    # Precision = TP/(TP+FP)
PREC = TP/(TP+FP)
    # False Positive Rate (FPR) = FP/(FP+TN)
FPR = FP/(FP+TN)
    # Accuracy = (TP+TN)/(TP+FP+TN+FN)
ACC = (TP+TN)/(TP+FP+TN+FN)
        # recall 
REC = TP/(TP+FN)

FAR = FP/(FP+TN)

        
    #F1=2*((Precision*Recall)/(Precision+Recall))
F1=2*((PREC*REC)/(PREC+REC))
print("Accuracy = \t" + str(ACC) +"\nFPR= \t "+ str(FPR) + "\nTPR= \t" + str(TPR)+ "\nPrecision= \t"+str(PREC)+"\nRecall= \t"+str(REC)+"\n F1= \t"+str(F1)+"\n FAR= \t"+str(FAR))
   
print("-------------------------------------------------------------------------------------------------------------------")

target_names = ['class 0', 'class 1', 'class 2','class3','class4']
#print("**************Built-In****************************")
#print(classification_report(y_true, y_pred, target_names=target_names))
#print("*************************************************").tp = tp.astype(float)
cl = 0;
for num in range(0,5):
    new_y_true = y_true.astype(int)
    new_y_pred = y_pred.astype(int)

    #Convert one class as 1 and rest of the classes as 0
    TP=0
    FP=0
    TN=0
    FN=0
    Total=0
    print("Class label",cl)
    for i in range(len(y_pred1)): 
        if y_true1[i]==y_pred1[i]==cl:
           TP += 1
        if y_pred1[i]!=cl and y_true1[i]!=y_pred1[i]:
           FP += 1
        if y_true1[i]!=cl and y_pred1[i]==cl:
           TN += 1
           FN += 1
        
    #print(TP, FP, TN, FN)
    Total=(TP+TN)
    
    print(" TP:\t" + str(TP) + "\nTN:\t" + str(TN) + "\nFP:\t" + str(FP) + "\nFN:\t" + str(FN)+"\nTotal Labels\t"+ str(Total))
    Accuracy=TP/(TP+TN)
    print("Accuracy=",Accuracy)
    cl=cl+1

    